<a href="https://colab.research.google.com/github/hwangho-kim/Utility-OAC/blob/main/%ED%81%B4%EB%A6%B0%EB%A3%B8_HVAC_%EC%B5%9C%EC%A0%81%ED%99%94_%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split # TimeSeriesSplit 대신 사용 간소화
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler
# from scipy.optimize import minimize # 필요시 주석 해제

# --- 0. 유틸리티 함수 (예: 엔탈피 계산) ---
def calculate_humidity_ratio(temp_c, rel_hum_percent, p_atm_pa=101325):
    """
    온도와 상대습도로부터 습도비(절대습도)를 계산합니다.
    :param temp_c: 온도 (섭씨)
    :param rel_hum_percent: 상대습도 (%)
    :param p_atm_pa: 대기압 (Pa), 기본값 101325 Pa
    :return: 습도비 (kg_water/kg_dry_air)
    """
    if pd.isna(temp_c) or pd.isna(rel_hum_percent): return np.nan
    # 포화수증기압 (Buck's equation)
    p_ws_pa = 611.21 * np.exp((18.678 - temp_c / 234.5) * (temp_c / (257.14 + temp_c)))
    # 실제 수증기압
    p_w_pa = (rel_hum_percent / 100) * p_ws_pa
    # 습도비
    humidity_ratio = 0.621945 * p_w_pa / (p_atm_pa - p_w_pa)
    return humidity_ratio

def calculate_enthalpy(temp_c, humidity_ratio):
    """
    온도와 습도비로부터 엔탈피를 계산합니다.
    :param temp_c: 온도 (섭씨)
    :param humidity_ratio: 습도비 (kg_water/kg_dry_air)
    :return: 엔탈피 (kJ/kg_dry_air)
    """
    if pd.isna(temp_c) or pd.isna(humidity_ratio): return np.nan
    # 엔탈피 (kJ/kg 건공기)
    enthalpy = 1.006 * temp_c + humidity_ratio * (2501 + 1.86 * temp_c)
    return enthalpy

class CleanroomHVACOptimizer:
    def __init__(self, num_oacs=14, num_low_chillers=8, num_high_chillers=8, num_cooling_towers=9):
        self.num_oacs = num_oacs
        self.num_low_chillers = num_low_chillers
        self.num_high_chillers = num_high_chillers
        self.num_cooling_towers = num_cooling_towers

        # 각 OAC 모델 및 스케일러, 피처/타겟 이름 저장용 딕셔너리
        self.oac_scalers_X = {}
        self.oac_models = {}
        self.oac_feature_names = {}
        self.oac_target_names = {} # 예: {'OAC1_예열_개도율', ...}

        # 저온 냉동기 모델 관련
        self.low_ch_scaler_X = None
        self.low_ch_model = None
        self.low_ch_feature_names = []
        self.low_ch_target_names = [] # 예: '저온냉동기_메인차압압력', ...

        # 고온 냉동기 모델 관련
        self.high_ch_scaler_X = None
        self.high_ch_model = None
        self.high_ch_feature_names = []
        self.high_ch_target_names = [] # 예: '고온냉동기_메인차압압력', ...

        # 최적화를 위한 설비 대당 평균 용량 (데이터 기반 추정 또는 전문가 설정 필요)
        # 이 값들은 시스템 성능에 따라 사용자가 직접 입력하거나, 과거 데이터로부터 추정해야 합니다.
        # 단위는 예측되는 부하의 단위와 일치해야 합니다. (예: 코일 개도율 합계, 엔탈피 변화량 등)
        self.avg_load_per_low_chiller = 500  # 예시: 저온 냉동기 1대당 평균 부하 처리 능력 (단위: 코일 개도율 합계 등)
        self.avg_load_per_high_chiller = 600 # 예시: 고온 냉동기 1대당 평균 부하 처리 능력
        self.avg_heat_rejection_per_ct = 1000 # 예시: 냉각탑 1대당 평균 열 방출 능력

    def _prepare_oac_data_single(self, df_full, oac_id_num):
        """지정된 OAC ID에 대한 학습용 X, y 데이터를 준비합니다."""
        # 외기 엔탈피 계산 (df_full에 미리 계산되어 있다고 가정 또는 여기서 계산)
        if '외기_엔탈피' not in df_full.columns:
            df_full['외기_습도비'] = df_full.apply(lambda row: calculate_humidity_ratio(row['외기_온도'], row['외기_습도']), axis=1)
            df_full['외기_엔탈피'] = df_full.apply(lambda row: calculate_enthalpy(row['외기_온도'], row['외기_습도비']), axis=1)

        # OAC 컬럼명 접두사 (실제 데이터셋에 맞게 확인 필요)
        # 예시: 'OAC1_', 'OAC2_', ... 또는 사용자가 제공한 피처 목록의 패턴을 따라야 함
        # 사용자가 제공한 피처 목록을 기반으로 정확한 컬럼명을 사용해야 합니다.
        # 여기서는 일반적인 패턴 'OAC{}_'을 사용합니다.
        oac_col_prefix = f"OAC{oac_id_num}_" # 사용자의 피처 목록에 따라 이 접두사가 다를 수 있습니다.

        # 입력 피처 (OAC 모델용) - 사용자가 제공한 피처 목록을 기반으로 구성
        # 'FAN Hz'는 CMH를 대변하는 값으로 사용
        features = [
            '외기_온도', '외기_습도', '외기_엔탈피',
            f'{oac_col_prefix}예열_set_point',
            f'{oac_col_prefix}예냉_set_point',
            f'{oac_col_prefix}냉각_set_point',
            f'{oac_col_prefix}승온_set_point',
            f'{oac_col_prefix}가습_set_point', # 가습은 set point가 습도일 수도, 온도일 수도 있음. 여기서는 값으로 가정.
            f'{oac_col_prefix}FAN_Hz'
        ]
        # 출력 피처 (OAC 모델용) - 코일 개도율
        targets = [
            f'{oac_col_prefix}예열_개도율',
            f'{oac_col_prefix}예냉_개도율',
            f'{oac_col_prefix}냉각_개도율',
            f'{oac_col_prefix}승온_개도율',
            f'{oac_col_prefix}가습_개도율'
        ]

        # 실제 데이터프레임에 존재하는 컬럼만 선택 (중요)
        valid_features = [f for f in features if f in df_full.columns]
        valid_targets = [t for t in targets if t in df_full.columns]

        if not valid_features or not valid_targets or len(valid_targets) != 5: # 5개 코일 개도율 확인
            # print(f"경고: OAC {oac_id_num}의 학습에 필요한 일부 피처 또는 타겟 컬럼이 데이터에 없습니다. 건너<0xEB><0><0x84>니다.")
            # print(f"  요구 피처: {features}, 발견 피처: {valid_features}")
            # print(f"  요구 타겟: {targets}, 발견 타겟: {valid_targets}")
            return None, None, [], []

        X = df_full[valid_features].copy()
        y = df_full[valid_targets].copy()

        # 결측치 처리 (간단한 평균 대체, 실제로는 더 정교한 방법 고려)
        X.fillna(X.mean(), inplace=True)
        y.fillna(y.mean(), inplace=True) # 개도율은 0~100 사이 값이므로, 평균 대체가 적절하지 않을 수 있음

        return X, y, valid_features, valid_targets

    def train_oac_models(self, df_historical_data):
        """모든 OAC에 대한 개도율 예측 모델을 학습합니다."""
        print("OAC 모델 학습 시작...")
        temp_df = df_historical_data.copy() # 원본 데이터 변경 방지

        # 외기 엔탈피 계산 (전체 데이터에 대해 한 번만 수행)
        if '외기_엔탈피' not in temp_df.columns:
            temp_df['외기_습도비'] = temp_df.apply(lambda row: calculate_humidity_ratio(row['외기_온도'], row['외기_습도']), axis=1)
            temp_df['외기_엔탈피'] = temp_df.apply(lambda row: calculate_enthalpy(row['외기_온도'], row['외기_습도비']), axis=1)

        for i in range(1, self.num_oacs + 1):
            # print(f"  OAC {i} 모델 학습 중...")
            X_oac, y_oac, features, targets = self._prepare_oac_data_single(temp_df, i)

            if X_oac is None or X_oac.empty or y_oac is None or y_oac.empty:
                print(f"  OAC {i} 데이터 준비 실패. 건너<0xEB><0><0x84>니다.")
                continue

            self.oac_feature_names[i] = features
            self.oac_target_names[i] = targets # 각 OAC별 타겟 컬럼명 저장

            scaler = StandardScaler()
            X_oac_scaled = scaler.fit_transform(X_oac)
            self.oac_scalers_X[i] = scaler

            # 시계열 데이터이므로 shuffle=False 옵션 중요 (또는 TimeSeriesSplit 사용)
            X_train, _, y_train, _ = train_test_split(X_oac_scaled, y_oac, test_size=0.2, shuffle=False, random_state=42)

            # 모델 정의 및 학습 (RandomForest 사용, n_estimators 등은 튜닝 필요)
            # n_jobs=-1은 모든 CPU 코어 사용
            model = MultiOutputRegressor(RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1, max_depth=15, min_samples_leaf=3))
            model.fit(X_train, y_train)
            self.oac_models[i] = model
        print("OAC 모델 학습 완료.")

    def predict_oac_coil_opening_ratios(self, current_outdoor_conditions_dict, oac_setpoints_list_of_dicts, modified_coil_info=None):
        """
        모든 OAC의 코일 개도율을 예측합니다.
        :param current_outdoor_conditions_dict: dict, {'외기_온도': 값, '외기_습도': 값}
        :param oac_setpoints_list_of_dicts: list of dicts, 각 OAC별 설정값.
               [{'OAC1_예열_set_point': 값, ...}, {'OAC2_예열_set_point': 값, ...}]
        :param modified_coil_info: dict, {'oac_id_num': int, 'coil_target_name': str, 'value': float}
               예: {'oac_id_num': 1, 'coil_target_name': 'OAC1_냉각_개도율', 'value': 30.0}
        :return: dict, {oac_id_num: {coil_name: predicted_value, ...}, ...}
        """
        all_oac_predictions = {}

        # 외기 조건으로부터 엔탈피 계산
        outdoor_temp = current_outdoor_conditions_dict['외기_온도']
        outdoor_humidity = current_outdoor_conditions_dict['외기_습도']
        outdoor_hr = calculate_humidity_ratio(outdoor_temp, outdoor_humidity)
        outdoor_enthalpy = calculate_enthalpy(outdoor_temp, outdoor_hr)

        current_outdoor_df_dict = {
            '외기_온도': outdoor_temp,
            '외기_습도': outdoor_humidity,
            '외기_엔탈피': outdoor_enthalpy
        }

        for i in range(1, self.num_oacs + 1):
            if i not in self.oac_models or i not in self.oac_scalers_X:
                # print(f"OAC {i} 모델이 학습되지 않았거나 스케일러가 없습니다. 예측을 건너<0xEB><0><0x84>니다.")
                # 타겟 이름이 저장되어 있다면 NaN으로 채움
                target_names_for_oac = self.oac_target_names.get(i, [f"OAC{i}_코일{j}_개도율" for j in range(1,6)])
                all_oac_predictions[i] = {name: np.nan for name in target_names_for_oac}
                continue

            # 현재 OAC의 설정값과 외기 조건을 결합
            current_oac_input_dict = current_outdoor_df_dict.copy()
            # oac_setpoints_list_of_dicts는 0-indexed
            if i-1 < len(oac_setpoints_list_of_dicts):
                 current_oac_input_dict.update(oac_setpoints_list_of_dicts[i-1])
            else:
                # print(f"경고: OAC {i}에 대한 설정값이 제공되지 않았습니다.")
                # 피처 이름에 해당하는 키가 없으면 NaN으로 채워짐
                pass


            # 학습 시 사용된 피처 순서대로 입력 데이터프레임 생성
            input_data_values = [current_oac_input_dict.get(fn, np.nan) for fn in self.oac_feature_names[i]]
            input_df = pd.DataFrame([input_data_values], columns=self.oac_feature_names[i])

            # 결측치 처리 (학습 시와 동일한 방식 또는 더 나은 방식)
            # 여기서는 간단히 평균값으로 채우지만, 실제로는 setpoint가 누락되면 예측이 어려움
            for col in input_df.columns:
                if input_df[col].isnull().any():
                    # 학습 데이터의 평균값으로 채우거나, 기본값 사용
                    # 여기서는 scaler에 저장된 평균을 사용할 수 없음. 임시로 0 또는 특정 값으로 대체.
                    # print(f"경고: OAC {i} 예측 입력값 중 {col}이 누락되었습니다. 0으로 대체합니다.")
                    input_df[col].fillna(0, inplace=True)


            input_scaled = self.oac_scalers_X[i].transform(input_df)
            prediction_array = self.oac_models[i].predict(input_scaled) # 결과는 (1, num_targets) 형태의 배열

            # 예측 결과를 {타겟명: 값} 형태의 딕셔너리로 변환
            oac_pred_dict = dict(zip(self.oac_target_names[i], prediction_array[0]))
            all_oac_predictions[i] = oac_pred_dict

        # 코일 개도율 수동 수정 시 반영 로직
        if modified_coil_info:
            mod_oac_id = modified_coil_info['oac_id_num']
            mod_coil_name = modified_coil_info['coil_target_name'] # 'OAC1_냉각_개도율' 같은 전체 타겟 이름
            mod_value = modified_coil_info['value']

            if mod_oac_id in all_oac_predictions and mod_coil_name in all_oac_predictions[mod_oac_id]:
                original_value = all_oac_predictions[mod_oac_id][mod_coil_name]
                all_oac_predictions[mod_oac_id][mod_coil_name] = mod_value
                print(f"알림: OAC {mod_oac_id}의 {mod_coil_name} 개도율이 {original_value:.2f}%에서 {mod_value:.2f}%(으)로 수동 변경되었습니다.")
                # --- 다른 코일 개도율 변화 반영 로직 (고급 기능) ---
                # 이 부분은 매우 복잡하며, 시스템의 물리적 모델 또는 추가적인 규칙/모델이 필요합니다.
                # 예시:
                # 1. 전체 엔탈피 변화량 목표를 유지하도록 다른 코일 조정
                # 2. 특정 코일(예: 냉각 코일) 변경 시, 연관된 코일(예: 재열 코일)을 규칙 기반으로 조정
                # 3. 또는, 이 변경이 시스템에 미치는 영향(예: 토출 온도 변화)을 사용자에게 알림
                print("      주의: 이 변경으로 인한 다른 코일들의 자동 연동 조정은 현재 단순 덮어쓰기 방식입니다.")
                print("      정교한 연동을 위해서는 추가적인 로직 개발이 필요합니다 (예: 에너지 균형, 제어 목표 기반 조정).")
            else:
                print(f"경고: 수정하려는 코일 정보(OAC ID: {mod_oac_id}, 코일명: {mod_coil_name})를 찾을 수 없습니다.")

        return all_oac_predictions

    def _prepare_chiller_data_common(self, df_full, oac_predictions_for_training, chiller_type):
        """냉동기 모델 학습을 위한 공통 데이터 준비 로직"""
        temp_df = df_full.copy()

        # 외기 엔탈피 계산 (이미 되어있다고 가정 또는 여기서 수행)
        if '외기_엔탈피' not in temp_df.columns and '외기_온도' in temp_df.columns and '외기_습도' in temp_df.columns:
             temp_df['외기_습도비'] = temp_df.apply(lambda row: calculate_humidity_ratio(row['외기_온도'], row['외기_습도']), axis=1)
             temp_df['외기_엔탈피'] = temp_df.apply(lambda row: calculate_enthalpy(row['외기_온도'], row['외기_습도비']), axis=1)


        # 냉동기 부하와 관련된 OAC 코일 개도율 합계 계산
        # chiller_type에 따라 '예냉_개도율' (고온) 또는 '냉각_개도율' (저온) 사용
        relevant_coil_suffix = '_예냉_개도율' if chiller_type == 'high' else '_냉각_개도율'
        total_relevant_coil_opening = pd.Series(0.0, index=temp_df.index)

        # oac_predictions_for_training은 학습 시에는 실제 과거 OAC 개도율 데이터,
        # 예측 시에는 OAC 모델의 예측값을 사용해야 합니다.
        # 이 함수는 학습용이므로, df_full (temp_df)에서 직접 OAC 개도율을 가져옵니다.
        for oac_id in range(1, self.num_oacs + 1):
            # 실제 OAC 개도율 컬럼명 확인 필요
            # 예: 'OAC1_예냉_개도율' 또는 'OAC1_냉각_개도율'
            oac_col_prefix = f"OAC{oac_id}_" # 이 부분은 실제 데이터 컬럼명 규칙에 따라야 함
            coil_col_name = f"{oac_col_prefix}{relevant_coil_suffix.lstrip('_')}" # lstrip으로 중복 '_' 제거

            if coil_col_name in temp_df.columns:
                # FAN_Hz (CMH 대리)를 가중치로 사용할 수 있음
                fan_hz_col = f"{oac_col_prefix}FAN_Hz"
                if fan_hz_col in temp_df.columns:
                    total_relevant_coil_opening += temp_df[coil_col_name].fillna(0) * temp_df[fan_hz_col].fillna(temp_df[fan_hz_col].mean()) # NaN 처리
                else:
                    total_relevant_coil_opening += temp_df[coil_col_name].fillna(0)
            # else:
                # print(f"경고: 냉동기({chiller_type}) 모델 학습 데이터 준비 중 {coil_col_name} 컬럼을 찾을 수 없습니다.")


        temp_df[f'Total_{chiller_type}_Coil_Load_Indicator'] = total_relevant_coil_opening

        # 입력 피처 (냉동기 모델용)
        features = [
            f'Total_{chiller_type}_Coil_Load_Indicator', # OAC 코일 부하 지표
            '외기_온도', # 외기 조건도 냉동기 부하에 영향
            '외기_습도',
            '외기_엔탈피' # 엔탈피도 포함 가능
        ]
        # 출력 피처 (냉동기 모델용 - 사용자가 제공한 피처 목록 기반)
        if chiller_type == 'low': # 저온 냉동기
            targets = ['저온냉동기_메인차압압력', '저온냉동기_메인차압개도율', '저온냉동기_서브차압개도율']
        else: # 고온 냉동기
            targets = ['고온냉동기_메인차압압력', '고온냉동기_메인차압개도율', '고온냉동기_서브차압개도율']

        valid_features = [f for f in features if f in temp_df.columns]
        valid_targets = [t for t in targets if t in temp_df.columns]

        if not valid_features or not valid_targets or len(valid_targets) != 3:
            # print(f"경고: {chiller_type} 냉동기 모델 학습에 필요한 피처 또는 타겟 컬럼이 부족합니다.")
            # print(f"  요구 피처: {features}, 발견 피처: {valid_features}")
            # print(f"  요구 타겟: {targets}, 발견 타겟: {valid_targets}")
            return None, None, [], []

        X = temp_df[valid_features].copy()
        y = temp_df[valid_targets].copy()
        X.fillna(X.mean(), inplace=True)
        y.fillna(y.mean(), inplace=True)

        return X, y, valid_features, valid_targets

    def train_chiller_models(self, df_historical_data):
        """저온 및 고온 냉동기 모델을 학습합니다."""
        print("냉동기 모델 학습 시작...")
        # 저온 냉동기 모델 학습
        # print("  저온 냉동기 모델 학습 중...")
        X_low, y_low, f_low, t_low = self._prepare_chiller_data_common(df_historical_data, None, 'low')
        if X_low is not None and not X_low.empty:
            self.low_ch_feature_names = f_low
            self.low_ch_target_names = t_low
            self.low_ch_scaler_X = StandardScaler().fit(X_low)
            X_low_scaled = self.low_ch_scaler_X.transform(X_low)

            X_train, _, y_train, _ = train_test_split(X_low_scaled, y_low, test_size=0.2, shuffle=False, random_state=42)
            model_low = MultiOutputRegressor(RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1, max_depth=10))
            model_low.fit(X_train, y_train)
            self.low_ch_model = model_low
        else:
            print("  저온 냉동기 모델 학습 데이터 준비 실패.")

        # 고온 냉동기 모델 학습
        # print("  고온 냉동기 모델 학습 중...")
        X_high, y_high, f_high, t_high = self._prepare_chiller_data_common(df_historical_data, None, 'high')
        if X_high is not None and not X_high.empty:
            self.high_ch_feature_names = f_high
            self.high_ch_target_names = t_high
            self.high_ch_scaler_X = StandardScaler().fit(X_high)
            X_high_scaled = self.high_ch_scaler_X.transform(X_high)

            X_train, _, y_train, _ = train_test_split(X_high_scaled, y_high, test_size=0.2, shuffle=False, random_state=42)
            model_high = MultiOutputRegressor(RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1, max_depth=10))
            model_high.fit(X_train, y_train)
            self.high_ch_model = model_high
        else:
            print("  고온 냉동기 모델 학습 데이터 준비 실패.")
        print("냉동기 모델 학습 완료.")

    def predict_chiller_parameters(self, all_oac_coil_predictions_dict, current_outdoor_conditions_dict):
        """
        OAC 코일 개도율 예측값을 기반으로 냉동기 파라미터를 예측합니다.
        :param all_oac_coil_predictions_dict: OAC 코일 개도율 예측 결과 (predict_oac_coil_opening_ratios의 반환값)
        :param current_outdoor_conditions_dict: 현재 외기 조건 dict
        :return: dict, {'low_temp_chiller': {param_name: value, ...}, 'high_temp_chiller': {param_name: value, ...}}
        """
        chiller_predictions_output = {'low_temp_chiller': {}, 'high_temp_chiller': {}}

        # 외기 조건으로부터 엔탈피 계산 (중복 계산 방지 위해 한번만)
        outdoor_temp = current_outdoor_conditions_dict['외기_온도']
        outdoor_humidity = current_outdoor_conditions_dict['외기_습도']
        outdoor_hr = calculate_humidity_ratio(outdoor_temp, outdoor_humidity)
        outdoor_enthalpy = calculate_enthalpy(outdoor_temp, outdoor_hr)
        current_outdoor_plus_enthalpy = {**current_outdoor_conditions_dict, '외기_엔탈피': outdoor_enthalpy}

        # 저온 냉동기 파라미터 예측
        if self.low_ch_model and self.low_ch_scaler_X:
            total_low_temp_load_indicator = 0
            for oac_id in range(1, self.num_oacs + 1):
                oac_preds = all_oac_coil_predictions_dict.get(oac_id, {})
                # 'OACx_냉각_개도율'과 같은 정확한 키 이름으로 접근해야 함
                # self.oac_target_names[oac_id] 에서 '냉각_개도율'을 포함하는 키를 찾아야 함
                cooling_coil_key = next((k for k in oac_preds.keys() if '냉각_개도율' in k), None)
                if cooling_coil_key:
                    fan_hz_key = next((k_sp for k_sp in self.oac_feature_names.get(oac_id, []) if 'FAN_Hz' in k_sp), None)
                    fan_hz_val = 1 # FAN_Hz 정보가 없거나 예측 입력에 없으면 기본값 1 (가중치 없음)
                    # FAN_Hz는 oac_setpoints_list_of_dicts 에서 가져와야 함. (여기서는 단순화를 위해 생략)
                    # 실제 구현 시에는 oac_setpoints_list_of_dicts 에서 해당 OAC의 FAN_Hz 값을 가져와야 함.
                    total_low_temp_load_indicator += oac_preds.get(cooling_coil_key, 0) * fan_hz_val

            input_data_low = {'Total_low_Coil_Load_Indicator': total_low_temp_load_indicator, **current_outdoor_plus_enthalpy}
            input_df_low_list = [input_data_low.get(fn, np.nan) for fn in self.low_ch_feature_names]
            input_df_low = pd.DataFrame([input_df_low_list], columns=self.low_ch_feature_names).fillna(0) # NaN은 0으로
            input_df_low_scaled = self.low_ch_scaler_X.transform(input_df_low)
            pred_low_array = self.low_ch_model.predict(input_df_low_scaled)
            chiller_predictions_output['low_temp_chiller'] = dict(zip(self.low_ch_target_names, pred_low_array[0]))
            chiller_predictions_output['low_temp_chiller']['calculated_total_load_indicator'] = total_low_temp_load_indicator # 부하 지표도 함께 반환

        # 고온 냉동기 파라미터 예측
        if self.high_ch_model and self.high_ch_scaler_X:
            total_high_temp_load_indicator = 0
            for oac_id in range(1, self.num_oacs + 1):
                oac_preds = all_oac_coil_predictions_dict.get(oac_id, {})
                precool_coil_key = next((k for k in oac_preds.keys() if '예냉_개도율' in k), None)
                if precool_coil_key:
                    # FAN_Hz 가중치 적용 (위와 동일하게 실제 구현 시 setpoint에서 가져와야 함)
                    total_high_temp_load_indicator += oac_preds.get(precool_coil_key, 0)

            input_data_high = {'Total_high_Coil_Load_Indicator': total_high_temp_load_indicator, **current_outdoor_plus_enthalpy}
            input_df_high_list = [input_data_high.get(fn, np.nan) for fn in self.high_ch_feature_names]
            input_df_high = pd.DataFrame([input_df_high_list], columns=self.high_ch_feature_names).fillna(0)
            input_df_high_scaled = self.high_ch_scaler_X.transform(input_df_high)
            pred_high_array = self.high_ch_model.predict(input_df_high_scaled)
            chiller_predictions_output['high_temp_chiller'] = dict(zip(self.high_ch_target_names, pred_high_array[0]))
            chiller_predictions_output['high_temp_chiller']['calculated_total_load_indicator'] = total_high_temp_load_indicator

        return chiller_predictions_output

    def optimize_chiller_operating_units(self, chiller_parameter_predictions):
        """
        예측된 냉동기 부하 지표를 기반으로 최적 가동 대수를 산출합니다.
        :param chiller_parameter_predictions: predict_chiller_parameters의 반환값.
                                            내부에 'calculated_total_load_indicator' 키가 있어야 함.
        :return: dict, {'optimal_low_temp_chillers': int, 'optimal_high_temp_chillers': int}
        """
        # 저온 냉동기 최적 대수
        low_ch_load = chiller_parameter_predictions.get('low_temp_chiller', {}).get('calculated_total_load_indicator', 0)
        num_optimal_low = 0
        if pd.notna(low_ch_load) and self.avg_load_per_low_chiller > 0:
            num_optimal_low = int(np.ceil(low_ch_load / self.avg_load_per_low_chiller))
        num_optimal_low = min(max(0, num_optimal_low), self.num_low_chillers) # 0 ~ 최대 대수 범위

        # 고온 냉동기 최적 대수
        high_ch_load = chiller_parameter_predictions.get('high_temp_chiller', {}).get('calculated_total_load_indicator', 0)
        num_optimal_high = 0
        if pd.notna(high_ch_load) and self.avg_load_per_high_chiller > 0:
            num_optimal_high = int(np.ceil(high_ch_load / self.avg_load_per_high_chiller))
        num_optimal_high = min(max(0, num_optimal_high), self.num_high_chillers)

        return {
            "optimal_low_temp_chillers_count": num_optimal_low,
            "optimal_high_temp_chillers_count": num_optimal_high
        }

    def optimize_cooling_tower_operating_units(self, optimal_chiller_counts_dict):
        """
        운전 중인 냉동기 대수와 부하를 고려하여 냉각탑 최적 가동 대수를 산출합니다.
        :param optimal_chiller_counts_dict: optimize_chiller_operating_units의 반환값.
        :return: dict, {'optimal_cooling_towers_count': int}
        """
        # 냉동기 총 부하 (열 방출량 추정)
        # 각 냉동기가 평균 용량만큼 가동된다고 가정하거나, 실제 예측된 부하량을 사용.
        # 여기서는 가동 대수 * 평균 용량으로 단순화.
        # 실제로는 냉동기의 COP(성능계수)를 고려하여 열 방출량 계산 (Q_reject = Q_evap + W_comp)
        # Q_evap (냉동 부하)는 chiller_parameter_predictions의 calculated_total_load_indicator 사용 가능.
        # W_comp (압축기 일) 데이터는 없으므로, Q_reject = Q_evap * (1 + 1/COP_assumed) 형태로 추정.
        # COP는 보통 3~5 사이. 여기서는 단순화를 위해 총 부하 지표를 사용.

        total_heat_rejection_load_indicator = (
            optimal_chiller_counts_dict["optimal_low_temp_chillers_count"] * self.avg_load_per_low_chiller +
            optimal_chiller_counts_dict["optimal_high_temp_chillers_count"] * self.avg_load_per_high_chiller
        )
        # 냉동기 부하 -> 냉각탑 방열 부하 변환 계수 (예: 1.25는 일반적인 값)
        total_heat_rejection_load_indicator *= 1.25

        num_optimal_ct = 0
        if pd.notna(total_heat_rejection_load_indicator) and self.avg_heat_rejection_per_ct > 0:
            num_optimal_ct = int(np.ceil(total_heat_rejection_load_indicator / self.avg_heat_rejection_per_ct))
        num_optimal_ct = min(max(0, num_optimal_ct), self.num_cooling_towers)

        return {"optimal_cooling_towers_count": num_optimal_ct}

# --- 메인 실행 로직 (예시) ---
if __name__ == '__main__':
    # 중요: 실제 데이터 파일 경로 및 컬럼명은 사용자의 환경에 맞게 수정해야 합니다.
    # data_file_path = "실제_6개월치_데이터.csv"
    # try:
    #     df_historical = pd.read_csv(data_file_path, parse_dates=['Datetime'])
    #     df_historical.set_index('Datetime', inplace=True)
    # except FileNotFoundError:
    #     print(f"오류: 데이터 파일 '{data_file_path}'을 찾을 수 없습니다. 더미 데이터를 생성합니다.")
    #     # 더미 데이터 생성 (실제 데이터 컬럼 구조와 최대한 유사하게)
    num_rows_dummy = 5000 # 학습 데이터 양 증가
    datetime_dummy = pd.to_datetime(pd.date_range(start='2023-01-01', periods=num_rows_dummy, freq='H')) # 시간 단위로 변경
    data_dummy = {'Datetime': datetime_dummy}
    # 외기
    data_dummy['외기_온도'] = np.random.uniform(0, 35, num_rows_dummy) # 겨울철 포함
    data_dummy['외기_습도'] = np.random.uniform(20, 95, num_rows_dummy)

    # OAC (14대) - 사용자가 제공한 피처 목록 기반
    for i in range(1, 15): # 1부터 14까지
        p = f"OAC{i}_" # 컬럼명 접두사 (실제 데이터와 일치 필요)
        data_dummy[f'{p}토출_온도'] = np.random.uniform(20, 25, num_rows_dummy)
        data_dummy[f'{p}토출_노점_온도'] = np.random.uniform(10, 15, num_rows_dummy)
        data_dummy[f'{p}토출_압력'] = np.random.uniform(100, 200, num_rows_dummy)
        data_dummy[f'{p}FAN_Hz'] = np.random.uniform(30, 60, num_rows_dummy)

        # 각 코일별 set point, 후단 온도/습도, 개도율
        for coil_type in ['예열', '예냉', '냉각', '승온']:
            data_dummy[f'{p}{coil_type}_set_point'] = np.random.uniform(10, 30, num_rows_dummy)
            data_dummy[f'{p}{coil_type}_후단_온도'] = data_dummy[f'{p}{coil_type}_set_point'] + np.random.uniform(-2, 2, num_rows_dummy)
            data_dummy[f'{p}{coil_type}_후단_습도'] = np.random.uniform(30, 70, num_rows_dummy) # 후단 습도는 복잡, 단순화
            data_dummy[f'{p}{coil_type}_개도율'] = np.random.uniform(0, 100, num_rows_dummy)

        data_dummy[f'{p}가습_set_point'] = np.random.uniform(40, 60, num_rows_dummy) # 가습 set point (습도 % 가정)
        data_dummy[f'{p}가습_개도율'] = np.random.uniform(0, 100, num_rows_dummy)

    # 저온 냉동기 (8대) - 메인 장비 및 개별 장비 데이터
    for prefix_type in [("저온냉동기", 8), ("고온냉동기", 8)]:
        main_p = prefix_type[0] + "_"
        data_dummy[f'{main_p}메인차압압력'] = np.random.uniform(0.5, 2.0, num_rows_dummy)
        data_dummy[f'{main_p}메인차압개도율'] = np.random.uniform(10, 90, num_rows_dummy)
        data_dummy[f'{main_p}서브차압개도율'] = np.random.uniform(10, 90, num_rows_dummy)
        data_dummy[f'{main_p}메인supply압력'] = np.random.uniform(3, 7, num_rows_dummy)
        data_dummy[f'{main_p}메인supply온도'] = np.random.uniform(5, 10, num_rows_dummy) if "저온" in main_p else np.random.uniform(10, 15, num_rows_dummy)
        data_dummy[f'{main_p}메인return압력'] = data_dummy[f'{main_p}메인supply압력'] - np.random.uniform(0.5, 1.5, num_rows_dummy)
        data_dummy[f'{main_p}메인return온도'] = data_dummy[f'{main_p}메인supply온도'] + np.random.uniform(3, 7, num_rows_dummy)

        # for j in range(1, prefix_type[1] + 1): # 개별 냉동기 데이터 (사용자가 제공한 피처 목록에 따라 추가)
        #     ch_p = f"{prefix_type[0]}{j}_"
        #     data_dummy[f'{ch_p}supply냉수온도'] = data_dummy[f'{main_p}메인supply온도'] + np.random.uniform(-0.5, 0.5, num_rows_dummy)
        #     # ... 나머지 개별 냉동기 데이터 ...

    # 냉각탑 (9대)
    # for k in range(1, 10):
    #     ct_p = f"냉각탑{k}_"
    #     data_dummy[f'{ct_p}supply냉각수온도'] = np.random.uniform(25, 35, num_rows_dummy)
    #     # ... 나머지 냉각탑 데이터 ...

    df_historical = pd.DataFrame(data_dummy)
    df_historical.set_index('Datetime', inplace=True)
    # --- 더미 데이터 생성 완료 ---

    # 시스템 옵티마이저 클래스 인스턴스 생성
    hvac_optimizer = CleanroomHVACOptimizer()

    # 1. 모델 학습 (충분한 양의 실제 데이터로 학습 필요)
    print("\n=== 모델 학습 시작 ===")
    hvac_optimizer.train_oac_models(df_historical)
    hvac_optimizer.train_chiller_models(df_historical) # OAC 개도율 데이터가 df_historical에 포함되어 있어야 함
    print("=== 모델 학습 완료 ===\n")

    # --- 예측 및 최적화 실행 (예시 시나리오) ---
    print("=== 예측 및 최적화 실행 ===")
    # 현재 외기 조건
    current_outdoor_conditions = {'외기_온도': 15.0, '외기_습도': 55.0} # 예: 봄/가을철 외기

    # OAC별 현재 설정값 (14개 OAC에 대한 설정값 리스트)
    # 각 OAC의 실제 컬럼명 접두사를 사용해야 함 (더미 데이터 생성 시 'OAC{i}_' 사용)
    example_oac_setpoints_input = []
    for i in range(1, hvac_optimizer.num_oacs + 1):
        oac_sp_prefix = f"OAC{i}_"
        example_oac_setpoints_input.append({
            f'{oac_sp_prefix}예열_set_point': 20.0,
            f'{oac_sp_prefix}예냉_set_point': 18.0,
            f'{oac_sp_prefix}냉각_set_point': 16.0, # 냉방 부하 약간 있는 상황
            f'{oac_sp_prefix}승온_set_point': 22.0,
            f'{oac_sp_prefix}가습_set_point': 50.0, # %RH 가정
            f'{oac_sp_prefix}FAN_Hz': 45.0
        })

    # 2. OAC 코일 개도율 예측
    print("\n--- 2. OAC 코일 개도율 예측 ---")
    predicted_oac_coils = hvac_optimizer.predict_oac_coil_opening_ratios(
        current_outdoor_conditions,
        example_oac_setpoints_input
    )
    # 결과 중 첫번째 OAC만 출력 (예시)
    if 1 in predicted_oac_coils and predicted_oac_coils[1]:
        print("OAC 1 예측된 코일 개도율:")
        for coil_name, value in predicted_oac_coils[1].items():
            print(f"  {coil_name}: {value:.2f}%")
    else:
        print("OAC 1 예측 결과 없음.")

    # (선택적) 특정 코일 개도율 수동 변경 후 재확인
    manual_modification = {'oac_id_num': 1, 'coil_target_name': 'OAC1_냉각_개도율', 'value': 40.0} # OAC1의 냉각코일 개도율을 40%로 수동 변경
    print(f"\n--- OAC {manual_modification['oac_id_num']}의 {manual_modification['coil_target_name']} 수동 변경 ({manual_modification['value']}%) 후 재예측 ---")
    predicted_oac_coils_modified = hvac_optimizer.predict_oac_coil_opening_ratios(
        current_outdoor_conditions,
        example_oac_setpoints_input,
        modified_coil_info=manual_modification
    )
    if 1 in predicted_oac_coils_modified and predicted_oac_coils_modified[1]:
        print(f"OAC {manual_modification['oac_id_num']} 수정 후 예측된 코일 개도율:")
        for coil_name, value in predicted_oac_coils_modified[1].items():
            print(f"  {coil_name}: {value:.2f}%")


    # 3. 냉동기 파라미터 예측 (수동 변경된 OAC 개도율 사용)
    print("\n--- 3. 냉동기 파라미터 예측 ---")
    predicted_chiller_params = hvac_optimizer.predict_chiller_parameters(
        predicted_oac_coils_modified, # 수동 변경된 OAC 예측값 사용
        current_outdoor_conditions
    )
    print("예측된 냉동기 파라미터:")
    if predicted_chiller_params['low_temp_chiller']:
        print(f"  저온 냉동기: {predicted_chiller_params['low_temp_chiller']}")
    else:
        print("  저온 냉동기: 예측 결과 없음")
    if predicted_chiller_params['high_temp_chiller']:
        print(f"  고온 냉동기: {predicted_chiller_params['high_temp_chiller']}")
    else:
        print("  고온 냉동기: 예측 결과 없음")


    # 4. 냉동기 최적 가동 대수 산출
    print("\n--- 4. 냉동기 최적 가동 대수 산출 ---")
    optimal_chiller_units = hvac_optimizer.optimize_chiller_operating_units(predicted_chiller_params)
    print(f"  최적 저온 냉동기 가동 대수: {optimal_chiller_units['optimal_low_temp_chillers_count']} 대 (최대 {hvac_optimizer.num_low_chillers}대)")
    print(f"  최적 고온 냉동기 가동 대수: {optimal_chiller_units['optimal_high_temp_chillers_count']} 대 (최대 {hvac_optimizer.num_high_chillers}대)")

    # 5. 냉각탑 최적 가동 대수 산출
    print("\n--- 5. 냉각탑 최적 가동 대수 산출 ---")
    optimal_ct_units = hvac_optimizer.optimize_cooling_tower_operating_units(optimal_chiller_units)
    print(f"  최적 냉각탑 가동 대수: {optimal_ct_units['optimal_cooling_towers_count']} 대 (최대 {hvac_optimizer.num_cooling_towers}대)")
    print("\n=== 예측 및 최적화 완료 ===")